### Use Python3 kernel for this notebook

In [ ]:
!scala -version

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
 .appName('DATA228 Spark & BigQuery Demo') \
 .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.17.3') \
 .getOrCreate()

In [ ]:
table = "bigquery-public-data.wikipedia.pageviews_2020"

df_wiki_pageviews = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .option("filter", "datehour >= '2020-10-01' AND datehour < '2020-10-02'") \
  .load()

df_wiki_pageviews.printSchema()

In [ ]:
df_wiki_pageviews.show()

In [ ]:
df_wiki_pageviews.select("title").show()

In [ ]:
df_wiki_en = df_wiki_pageviews \
  .select("datehour", "wiki", "views") \
  .where("views > 1000 AND wiki == 'en'") \

df_wiki_en.printSchema()

In [ ]:
df_wiki_en.show()

In [ ]:
df_wiki_en.count()

In [ ]:
df_wiki_en_by_hour = df_wiki_en.groupby("datehour")

In [ ]:
type(df_wiki_en_by_hour)

In [ ]:
import pyspark.sql.functions as sql_func
df_date_hour_totals = df_wiki_en_by_hour.agg(sql_func.sum('views').alias('total_views'))

In [ ]:
df_date_hour_totals.show()

In [ ]:
df_date_hour_totals.orderBy('total_views', ascending=False).show(30)

In [ ]:
df_date_hour_totals.coalesce(1).write.csv("gs://echan-pyspark/df_data_hour_totals.csv")

In [ ]:
!gsutil ls gs://echan-pyspark/df_data_hour_totals.csv

In [ ]:
!hdfs dfs -ls gs://echan-pyspark/df_data_hour_totals.csv